In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

def get_completion(prompt):
    response = client.chat.completions.create(
    model="llama3",
    temperature=0,
    max_tokens=1000,
    messages=[
        {"role": "user", "content": prompt}
    ])
    return response.choices[0].message.content

get_completion('Hello.')

with open('../prompts/prompt-current.txt', 'r') as f:
    prompt = f.read()
print(prompt)